In [55]:
import pandas as pd  
import sqlite3


conn = sqlite3.connect("database.sqlite")


df_teams = pd.read_sql_query("SELECT * FROM team;", conn)
df_countries = pd.read_sql_query("SELECT * FROM country;", conn)
df_matchs = pd.read_sql_query("SELECT * FROM match;", conn)
df_leagues = pd.read_sql_query("SELECT * FROM league;", conn)
df_player = pd.read_sql_query("SELECT * FROM player;", conn)
df_player_attribute = pd.read_sql_query("SELECT * FROM Player_Attributes;", conn)
df_teams_attributes=pd.read_sql_query("select * from team_attributes;",conn)

df_matchs=df_matchs.drop(columns=['B365H','B365D','B365A','BWH','BWD','BWA','IWH','IWD','IWA','LBH','LBD','LBA','PSH','PSD','PSA','WHH','WHD','WHA','SJH','SJD','SJA','VCH','VCD','VCA','GBH','GBD','GBA','BSH','BSD','BSA'])
print(df_matchs.columns)

conn.close()


Index(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal', 'away_team_goal', 'home_player_X1', 'home_player_X2',
       'home_player_X3', 'home_player_X4', 'home_player_X5', 'home_player_X6',
       'home_player_X7', 'home_player_X8', 'home_player_X9', 'home_player_X10',
       'home_player_X11', 'away_player_X1', 'away_player_X2', 'away_player_X3',
       'away_player_X4', 'away_player_X5', 'away_player_X6', 'away_player_X7',
       'away_player_X8', 'away_player_X9', 'away_player_X10',
       'away_player_X11', 'home_player_Y1', 'home_player_Y2', 'home_player_Y3',
       'home_player_Y4', 'home_player_Y5', 'home_player_Y6', 'home_player_Y7',
       'home_player_Y8', 'home_player_Y9', 'home_player_Y10',
       'home_player_Y11', 'away_player_Y1', 'away_player_Y2', 'away_player_Y3',
       'away_player_Y4', 'away_player_Y5', 'away_player_Y6', 'away_player_Y7',
       'away_player_Y8', 'away_

Création d’une Vue Consolidée : Combinez les informations des tables Player, Player_Attributes, et Team pour créer une base de données unifiée qui relie les joueurs à leurs équipes et performances.


In [56]:
player_team=df_player.merge(df_teams)
player_team_att=player_team.merge(df_player_attribute,on='id').reset_index()
full_team=pd.merge(player_team,df_teams)
full_team.set_index(keys='id')

df_matchs


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,NaN,NaN,None,None,None,None,None,None,None,None
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,NaN,NaN,None,None,None,None,None,None,None,None
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,NaN,NaN,None,None,None,None,None,None,None,None
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,NaN,NaN,None,None,None,None,None,None,None,None
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,NaN,NaN,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,232110.0,289732.0,None,None,None,None,None,None,None,None
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,34082.0,95257.0,None,None,None,None,None,None,None,None
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,451335.0,275122.0,None,None,None,None,None,None,None,None
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,92252.0,194532.0,None,None,None,None,None,None,None,None


cree fonction pour extracter les donnes des **goals** au session on  xml 

In [57]:
import xml.etree.ElementTree as ET
import pandas as pd
from collections import defaultdict
import pandas as pd

def store_goals_to_dataframe(xml_data):
    
    
    # Parse the XML data
    root = ET.fromstring(xml_data)
    goals_list = []

    # Iterate over each 'value' element inside 'goal'
    for value in root.findall('value'):
        goal_data = defaultdict(lambda: None)  # Default to None if key is missing

        # Extract elements, checking for None before accessing .text
        goal_data['goal_id'] = int(value.find('id').text) if value.find('id') is not None else None
        goal_data['team'] = int(value.find('team').text) if value.find('team') is not None else None
        goal_data['player1'] = int(value.find('player1').text) if value.find('player1') is not None else None
        goal_data['player2'] = int(value.find('player2').text) if value.find('player2') is not None else None
        goal_data['elapsed'] = int(value.find('elapsed').text) if value.find('elapsed') is not None else None
        goal_data['goal_type'] = value.find('goal_type').text if value.find('goal_type') is not None else None
        goal_data['goal_subtype'] = value.find('subtype').text if value.find('subtype') is not None else None
        goal_data['goal_comment'] = value.find('comment').text if value.find('comment') is not None else None
        goal_data['event_type'] = int(value.find('event_incident_typefk').text) if value.find('event_incident_typefk') is not None else None
        goal_data['sort_order'] = int(value.find('sortorder').text) if value.find('sortorder') is not None else None
        goal_data['type_goal'] = value.find('type').text if value.find('type') is not None else None
        goal_data['stats'] = value.find('stats')  # This can be None if not present
        
        # extracter  goal from stats 
        if goal_data['stats'] is not None:
            goal_data['goals'] = int(goal_data['stats'].find('goals').text) if goal_data['stats'].find('goals') is not None else None
            # goal_data['shoton'] = int(goal_data['stats'].find('shoton').text) if goal_data['stats'].find('shoton') is not None else None
        else:
            goal_data['goals'] = None
            # goal_data['shoton'] = None

        # # Append goal data to the list
        goals_list.append(goal_data)

    # Convert the list of dictionaries to a DataFrame
    goal_df = pd.DataFrame(goals_list)

    return goal_df


season1=df_matchs[df_matchs['season']=='2015/2016']

df_without_Nan = season1.dropna(subset=['goal'])
l = df_without_Nan['goal']
all_goals_df = []

for index, value in enumerate(l):
    goal_df = store_goals_to_dataframe(value)
    all_goals_df.append(goal_df)

final_goals_df = pd.concat(all_goals_df, ignore_index=True)





/tmp/ipykernel_5916/3327009174.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_goals_df = pd.concat(all_goals_df, ignore_index=True)


<!-- SHOOTER PLAYER -->

shot on : saved by the goalkeeper or blocked by a defender while heading towards the goal.

shot off : not directed toward the goal entirelly
player1: is player who scored goal
player2 is player who assist it 

In [59]:


final_goals_df=final_goals_df.rename(columns={'player1': 'player_api_id'}, inplace=False)

merged=final_goals_df.merge(df_player)
# print(shoton.columns)
# merged=merged.merge(shots_per_player,on='player_api_id')
# print(shots_per_player.columns())
merged_players_attribute=merged.merge(df_player_attribute,on='id')

plus_performants_joueurs=merged_players_attribute.groupby(['player_name']).agg({
 'goals':'sum',
  'potential':'mean',
 'overall_rating':'mean',
 'finishing':'sum',
 'dribbling':'mean',
 'sprint_speed':'mean'
 
 }).reset_index().sort_values(by=['goals','potential','overall_rating','finishing','dribbling','sprint_speed'],ascending=False)
# # print(f"=> most goals makers on season 2016/2015: \n ")
plus_performants_joueurs


,player_name,goals,potential,overall_rating,finishing,dribbling,sprint_speed
978,Luis Suarez,40.0,73.0,67.0,546.0,18.0,43.0
1689,Zlatan Ibrahimovic,38.0,77.0,72.0,2132.0,62.0,61.0
606,Gonzalo Higuain,36.0,75.0,69.0,1224.0,62.0,69.0
324,Cristiano Ronaldo,35.0,65.0,65.0,2496.0,54.0,55.0
1353,Robert Lewandowski,30.0,74.0,74.0,775.0,25.0,42.0
...,...,...,...,...,...,...,...
1072,Marvin Peersman,0.0,59.0,59.0,12.0,53.0,31.0
358,Danilo Fernando Avelar,0.0,59.0,51.0,25.0,30.0,62.0
258,Carlitos,0.0,57.0,51.0,32.0,44.0,66.0
483,Etienne Reijnen,0.0,53.0,49.0,46.0,51.0,55.0


Identifiez les joueurs les plus constants en performance au fil des saisons.

In [ ]:

final_goals_df=final_goals_df.rename(columns={'player1': 'player_api_id'}, inplace=False)
merged=final_goals_df.merge(df_player)
# merged=merged.merge(shots_per_player,on='player_api_id')
# print(shots_per_player.columns())
print(df_leagues)
merged_players_attribute=merged.merge(df_player_attribute,on='id')
print(df_matchs.columns)
# goals_league=pd.merge(goals_teams,df_leagues,left_on='id',right_on='league_id')
# goals_league.rename({'name':'league_name'})



==================================================================================================================================
Analysez les victoires, défaites, et matchs nuls pour identifier les équipes les plus efficaces.

Comparez les équipes en termes de styles de jeu : vitesse de construction, précision des passes, et pression défensive.

In [ ]:
goals_teams = pd.merge(final_goals_df, df_teams, left_on='team', right_on='team_api_id', how='inner')
# goals_teams_league=pd.merge(goals_teams,df_leagues)
goals_teams_merged=goals_teams.groupby(['team_long_name'])['goals'].sum().reset_index().sort_values(by='goals',ascending=False)

print("les équipes dominent dans chaque ligue ")
goals_teams_merged

final_goals_df


In [ ]:
df_matchs['possession']
match=df_matchs.dropna(subset='possession')



# import pandas as pd
import xml.etree.ElementTree as ET



def extract_possession(possession_xml):
  
    root = ET.fromstring(possession_xml)
    
    # Extract homepos and awaypos values from the XML structure
    homepos = None
    awaypos = None
    for value in root.findall('.//value'):  # Find all <value> tags
        homepos = value.find('homepos').text if value.find('homepos') is not None else None
        awaypos = value.find('awaypos').text if value.find('awaypos') is not None else None
    
    return homepos, awaypos

# Step 3: Apply the function to 'possession' column and create new columns for homepos and awaypos
match[['homepos', 'awaypos']] = match['possession'].apply(lambda x: pd.Series(extract_possession(x)))

# Step 4: Output the resulting dataframe with homepos and awaypos columns
print(match[['match_id', 'homepos', 'awaypos']])


In [ ]:
# match + team_home + team_away + country + league
leagues_match = pd.merge(df_matchs, df_leagues, 
                         left_on=['league_id', 'country_id'],  # left table columns
                         right_on=['id', 'country_id'],        # right table columns
                         how='inner')
leagues_match=leagues_match.rename({
    "name":"league_name"
})

match_leagues=leagues_match[['season','home_team_api_id','away_team_api_id','home_team_goal','name']]
# print(match_leagues.columns)
# goals_teams_merged=goals_teams.groupby(['name'])['goals'].sum().reset_index().sort_values(by='goals',ascending=False)

matchs_team_home = pd.merge(match_leagues, df_teams, left_on='home_team_api_id', right_on='team_api_id', how='inner')
matchs_team_home_away = pd.merge(matchs_team_home, df_teams, left_on='away_team_api_id', right_on='team_api_id', how='inner')

matchs_team_home_away.groupby([''])
# matchs_team_home_final=matchs_team_home_away.groupby(['name','team_long_name_x'])['home_team_goal'].sum()
# matchs_team_home_final


# matchs_team_home
# matchs_team_home
# # leagues_match.drop(columns=['id_leagues'], inplace=True)

# # a=matchs_team_home.groupby('team_long_name')['home_team_goal'].sum()
# matchs_team_home
# print(f"{match_leagues.columns}\n {df_teams.columns}")
# matchs_team_country=pd.merge(matchs_team_home_away,df_countries,le)


# merge_leagues
# matchs_team_home_away_league=pd.merge(matchs_team_home_away,df_leagues,left_on='league_id',right_on='id')
# matchs_team_home_away.columns

# print("les équipes dominent dans chaque ligue ")
# goals_teams_merged


,season,home_team_api_id,away_team_api_id,home_team_goal,name,id_x,team_api_id_x,team_fifa_api_id_x,team_long_name_x,team_short_name_x,id_y,team_api_id_y,team_fifa_api_id_y,team_long_name_y,team_short_name_y
0,2008/2009,9987,9993,1,Belgium Jupiler League,1,9987,673.0,KRC Genk,GEN,2,9993,675.0,Beerschot AC,BAC
1,2008/2009,10000,9994,0,Belgium Jupiler League,3,10000,15005.0,SV Zulte-Waregem,ZUL,4,9994,2007.0,Sporting Lokeren,LOK
2,2008/2009,9984,8635,0,Belgium Jupiler League,5,9984,1750.0,KSV Cercle Brugge,CEB,6,8635,229.0,RSC Anderlecht,AND
3,2008/2009,9991,9998,5,Belgium Jupiler League,7,9991,674.0,KAA Gent,GEN,8,9998,1747.0,RAEC Mons,MON
4,2008/2009,7947,9985,1,Belgium Jupiler League,9,7947,NaN,FCV Dender EH,DEN,10,9985,232.0,Standard de Liège,STL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,2015/2016,10190,10191,1,Switzerland Super League,49479,10190,898.0,FC St. Gallen,GAL,49837,10191,1715.0,FC Thun,THU
25975,2015/2016,9824,10199,1,Switzerland Super League,49122,9824,286.0,FC Vaduz,VAD,49121,10199,897.0,FC Luzern,LUZ
25976,2015/2016,9956,10179,2,Switzerland Super League,49115,9956,322.0,Grasshopper Club Zürich,GRA,49120,10179,110770.0,FC Sion,SIO
25977,2015/2016,7896,10243,0,Switzerland Super League,51606,7896,NaN,Lugano,LUG,49124,10243,894.0,FC Zürich,ZUR
